In [8]:
import numpy as np
import pandas as pd
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Step 1: Load the data
tesla_data = pd.read_csv('Clean_data/Cleaned_Tesla_Close.csv', parse_dates=['Date'], index_col='Date')
sp500_data = pd.read_csv('Clean_data/Cleaned_SP500_Close.csv', parse_dates=['Date'], index_col='Date')
ixic_data = pd.read_csv('Clean_data/Cleaned_IXIC_Close.csv', parse_dates=['Date'], index_col='Date')

# Merge datasets on 'Date'
merged_data = tesla_data.merge(sp500_data, left_index=True, right_index=True, suffixes=('', '_sp500'))
merged_data = merged_data.merge(ixic_data, left_index=True, right_index=True, suffixes=('', '_ixic'))

# Step 2: Preprocess the data
data = merged_data[['Close_predicted', 'Close_predicted_sp500', 'Close_predicted_ixic']]

# Scale the data after making it stationary
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.dropna())  # Ensure NaN values are removed after differencing
# Prepare the dataset for LSTM
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i - time_step:i])
        y.append(dataset[i, 0])  # Predict the 'Close' price of Tesla
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Function to create and train the LSTM model with Optuna
def objective(trial):
    # Hyperparameters to optimize
    lstm_units = trial.suggest_int('lstm_units', 50, 100)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 100, 200)

    # Build the LSTM model
    model = Sequential([
        Input(shape=(time_step, X_train.shape[2])),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units),
        Dropout(dropout_rate),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the model
    test_predictions = model.predict(X_test)
    test_predictions_rescaled = scaler.inverse_transform(
        np.hstack([test_predictions, X_test[:, -1, 1:]])
    )[:, 0]
    y_test_rescaled = scaler.inverse_transform(
        np.hstack([y_test.reshape(-1, 1), X_test[:, -1, 1:]])
    )[:, 0]

    rmse = np.sqrt(mean_squared_error(y_test_rescaled, test_predictions_rescaled))
    return rmse

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print('Best trial:')
trial = study.best_trial
print(f'  Value (RMSE): {trial.value}')
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2024-11-16 22:32:52,148] A new study created in memory with name: no-name-59e92c55-1572-4da3-9dab-bdc2a5f614df


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step


[I 2024-11-16 22:34:56,930] Trial 0 finished with value: 17.729190960506404 and parameters: {'lstm_units': 76, 'dropout_rate': 0.27746673263988897, 'learning_rate': 7.249980148144902e-05, 'batch_size': 16, 'epochs': 105}. Best is trial 0 with value: 17.729190960506404.


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step


[I 2024-11-16 22:37:24,928] Trial 1 finished with value: 16.64092801459754 and parameters: {'lstm_units': 78, 'dropout_rate': 0.3962576263628458, 'learning_rate': 0.002115787647684285, 'batch_size': 16, 'epochs': 124}. Best is trial 1 with value: 16.64092801459754.


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


[I 2024-11-17 00:42:32,908] Trial 48 finished with value: 23.418652233521716 and parameters: {'lstm_units': 73, 'dropout_rate': 0.37134918434171726, 'learning_rate': 0.0007564831485472273, 'batch_size': 16, 'epochs': 152}. Best is trial 36 with value: 15.389339236158593.


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


[I 2024-11-17 00:43:56,099] Trial 49 finished with value: 16.082128400629472 and parameters: {'lstm_units': 65, 'dropout_rate': 0.3161620354323499, 'learning_rate': 0.00046138384728697615, 'batch_size': 64, 'epochs': 163}. Best is trial 36 with value: 15.389339236158593.


Best trial:
  Value (RMSE): 15.389339236158593
  Params:
    lstm_units: 93
    dropout_rate: 0.3296526821355444
    learning_rate: 0.00035327026589794
    batch_size: 16
    epochs: 168


In [ ]:
import numpy as np
import pandas as pd
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Step 1: Load the data
tesla_data = pd.read_csv('Clean_data/Cleaned_Tesla_Close.csv', parse_dates=['Date'], index_col='Date')
sp500_data = pd.read_csv('Clean_data/Cleaned_SP500_Close.csv', parse_dates=['Date'], index_col='Date')
ixic_data = pd.read_csv('Clean_data/Cleaned_IXIC_Close.csv', parse_dates=['Date'], index_col='Date')

# Merge datasets on 'Date'
merged_data = tesla_data.merge(sp500_data, left_index=True, right_index=True, suffixes=('', '_sp500'))
merged_data = merged_data.merge(ixic_data, left_index=True, right_index=True, suffixes=('', '_ixic'))

# Step 2: Preprocess the data
data = merged_data[['Close_predicted', 'Close_predicted_sp500', 'Close_predicted_ixic']]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.dropna())  # Ensure NaN values are removed after differencing

# Prepare the dataset for LSTM
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i - time_step:i])
        y.append(dataset[i, 0])  # Predict the 'Close' price of Tesla
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Function to create and train the LSTM model with Optuna
def objective(trial):
    # Hyperparameters to optimize
    lstm_units = trial.suggest_int('lstm_units', 55, 60)
    dropout_rate = trial.suggest_float('dropout_rate', 0.265, 0.275)
    learning_rate = trial.suggest_float('learning_rate', 0.00095, 0.0011, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16])
    epochs = trial.suggest_int('epochs', 100, 200)

    # Build the LSTM model
    model = Sequential([
        Input(shape=(time_step, X_train.shape[2])),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units),
        Dropout(dropout_rate),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the model
    test_predictions = model.predict(X_test)
    test_predictions_rescaled = scaler.inverse_transform(
        np.hstack([test_predictions, X_test[:, -1, 1:]])
    )[:, 0]
    y_test_rescaled = scaler.inverse_transform(
        np.hstack([y_test.reshape(-1, 1), X_test[:, -1, 1:]])
    )[:, 0]

    rmse = np.sqrt(mean_squared_error(y_test_rescaled, test_predictions_rescaled))
    return rmse

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Print the best hyperparameters
print('Best trial:')
trial = study.best_trial
print(f'  Value (RMSE): {trial.value}')
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')
